In [116]:
from math import pi,cos,sin
import os
import pandas as pd
import time
import pickle
from multiprocessing import Pool
import multiprocessing as mp
from joblib import Parallel, delayed
from queue import PriorityQueue
import numpy as np
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()
inputs = myList



In [43]:
os.chdir('/home/lucasperea/Documents/Diffusion_MOF/')

In [161]:
# %store -r path_dic
%store path_dic
len(path_dic)

Stored 'path_dic' (dict)


1249

In [156]:
def funclon(x):
    y = x+2+5
    return y

if __name__ == "__main__":
    

    pool = mp.Pool(mp.cpu_count())

    result = pool.map_async(func=funclon, iterable=list(range(100)), callback=get_result)
    pool.close()
    pool.join()

In [189]:
def VolDist(file):
    temporal_dic = {}
    nt2_name = file
    print(nt2_name)

    vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
    G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

    final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
    Y, Y1 = TransfMatrix(vector_values_latt)
    final2 = distance(final,3, Y, Y1)



    neigh_dic = {}
    for n, nbrs in final2.adj.items():
        neigh = []
        for nbr, eattr in nbrs.items():
            neigh.append(nbr)


        neigh_dic[str(n)] = neigh

    all_path = []
    total_path = []
    special_list = list(range(13))

    larger_node = []
    name_larger_node = []
    for node in final.nodes:
        larger_node.append(final.nodes[node]['rad_max_sph'])
        name_larger_node.append(node)

    node_largest = name_larger_node[np.argmax(larger_node)]
    node_smallest = name_larger_node[np.argmin(larger_node)]
    node_path = node_largest #mat_pr_dic[nt2_name.split('.')[0]]

    for location in special_list:
        parents = {}

        start = node_path + key_plus*location

        target = key_plus*pos_order['000'] + node_path

        D, P = dijkstra(start, target, parents, final2, 0)
        try:
            path = backpedal(start, target, P)
        except:
            path = [node_smallest, neigh_dic[str(node_smallest)][0]]

        distance_path = []
        for i in range(len(path)-1):

            distance_path.append(final2[path[i]][path[i+1]]['distance'])


        all_path.append(path)
        total_path.append(sum(distance_path))  


    total_vol = []
    for ind_path in all_path:
        volume = []
        for i in range(len(ind_path) - 1):
            volume.append((final2[ind_path[i]][ind_path[i+1]]['distance'] * (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi))
    #                 ((final2[ind_path[i]][ind_path[i+1]]['distance'] - final2.nodes[ind_path[i]]['rad_max_sph'] - 
    #                   final2.nodes[ind_path[i+1]]['rad_max_sph'])*
    #                  (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi) + 
    #                 (final2.nodes[ind_path[i]]['rad_max_sph']**3)*(4/3)*pi +
    #                 (final2.nodes[ind_path[i+1]]['rad_max_sph']**3)*(4/3)*pi))
        total_vol.append(sum(volume))

    path_dic_mat = {}

    for j in range(len(list(pos_order.keys())[0:13])):
        path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


#     path_dic[nt2_name.split('.')[0]] = path_dic_mat
    
    temporal_dic[nt2_name.split('.')[0]] = path_dic_mat
    return temporal_dic

In [191]:
VolDist('DAXHUH01_clean' + '.nt2')

DAXHUH01_clean.nt2


KeyError: 141

In [120]:
def get_result(result):
    global results
    results.append(result)

In [184]:
os.path.getsize('./Nt2Files_MOF/QOYYOU_clean.nt2')/1e6

6.651255

In [185]:
# path_dic = {}
for file in os.listdir('./Nt2Files_MOF/'):

    nt2_name = file
    
    if os.path.getsize('./Nt2Files_MOF/' + file)/1e6 > 1:
        pass
    else:

        vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
        G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

        final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
        Y, Y1 = TransfMatrix(vector_values_latt)
        final2 = distance(final,3, Y, Y1)



        neigh_dic = {}
        for n, nbrs in final2.adj.items():
            neigh = []
            for nbr, eattr in nbrs.items():
                neigh.append(nbr)


            neigh_dic[str(n)] = neigh

        all_path = []
        total_path = []
        special_list = list(range(13))

        larger_node = []
        name_larger_node = []
        for node in final.nodes:
            larger_node.append(final.nodes[node]['rad_max_sph'])
            name_larger_node.append(node)

        node_largest = name_larger_node[np.argmax(larger_node)]
        node_smallest = name_larger_node[np.argmin(larger_node)]
        node_path = node_largest #mat_pr_dic[nt2_name.split('.')[0]]

        for location in special_list:
            parents = {}

            start = node_path + key_plus*location

            target = key_plus*pos_order['000'] + node_path

            D, P = dijkstra(start, target, parents, final2, 0)
            try:
                path = backpedal(start, target, P)
            except:
                path = [node_smallest, neigh_dic[str(node_smallest)][0]]

            distance_path = []
            for i in range(len(path)-1):

                distance_path.append(final2[path[i]][path[i+1]]['distance'])


            all_path.append(path)
            total_path.append(sum(distance_path))  


        total_vol = []
        for ind_path in all_path:
            volume = []
            for i in range(len(ind_path) - 1):

    #                     volume.append(final2[ind_path[i]][ind_path[i+1]]['distance']*
    #                      (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi)

                ((((final2[ind_path[i]][ind_path[i+1]]['distance']- final2.nodes[ind_path[i]]['rad_max_sph'] - 
                  final2.nodes[ind_path[i+1]]['rad_max_sph'])*
                 (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi) + 
            (final2.nodes[ind_path[i]]['rad_max_sph']**3)*(4/3)*pi +
            (final2.nodes[ind_path[i+1]]['rad_max_sph']**3)*(4/3)*pi))
            volume.append((final2[ind_path[i]][ind_path[i+1]]['distance'] * (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi))
            total_vol.append(sum(volume))

        path_dic_mat = {}

        for j in range(len(list(pos_order.keys())[0:13])):
            path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


        path_dic[nt2_name.split('.')[0]] = path_dic_mat
        print(nt2_name)

Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
Target vertex: 17105 reached
WAVQUH_clean.nt2
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
SONVAT_neutral_b.nt2
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 27302 reached
Target vertex: 273

Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
Target vertex: 15679 reached
ZISYAD_clean.nt2
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
Target vertex: 34949 reached
UWODAN_clean.nt2
Target vertex: 81677 reached
Target vertex: 81677 reached
Target vertex: 81677 reached
Target vertex: 81677 reached
Target vertex: 81677 reached
Target vertex: 81677 reached
Target vertex: 81677 reached
Target ve

Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
Target vertex: 24071 reached
LAHTUM_clean.nt2
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
Target vertex: 7304 reached
RESSAK_clean.nt2
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 reached
Target vertex: 12409 r

Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
Target vertex: 22929 reached
DOYBUQ_clean.nt2
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
Target vertex: 16954 reached
RAVWES_clean.nt2
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target vertex: 16829 reached
Target ve

Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
Target vertex: 22766 reached
MUHPAI_clean.nt2
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
Target vertex: 44952 reached
HISJIE_clean.nt2
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target vertex: 66655 reached
Target ve

...

Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
Target vertex: 10798 reached
ECAHOH_clean.nt2
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
Target vertex: 12620 reached
EKOPEA_clean.nt2
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target vertex: 18168 reached
Target ve

Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
Target vertex: 30691 reached
HUWSUN_clean.nt2
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
Target vertex: 63985 reached
QUNVIG_clean.nt2
Target vertex: 39059 reached
Target vertex: 39059 reached
Target vertex: 39059 reached
Target vertex: 39059 reached
Target vertex: 39059 reached
Target vertex: 39059 reached
Target vertex: 39059 reached
Target ve

Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
Target vertex: 15763 reached
KOFPEB_clean.nt2
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
Target vertex: 1516 reached
CEMYUQ_clean.nt2
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
Target vertex: 3830 reached
T

Target vertex: 53233 reached
Target vertex: 53233 reached


KeyboardInterrupt: 

In [187]:
path_dic2 = path_dic

In [188]:
%store path_dic2

Stored 'path_dic2' (dict)


In [170]:
len(path_dic)

1249

In [110]:
for i in range(len(all_path[4])-1):
    print(final2.nodes[all_path[4][i]]['rad_max_sph'],final2[all_path[4][i]][all_path[4][i+1]]['rad_max_sph'], final2.nodes[all_path[4][i]]['rad_max_sph'])

7.70855 7.52908 7.70855
7.52908 5.7993 7.52908
5.7993 3.00974 5.7993
5.7993 5.7993 5.7993
7.52908 7.52908 7.52908


In [105]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19},
 'VEXYON_charged': {'-1-1-1': 26.23,
  '-1-10': 24.22,
  '-1-11': 26.23,
  '-10-1': 24.22,
  '-100': 58.68,
  '-101': 24.22,
  '-11-1': 26.23,
  '-110': 24.22,
  '-111': 26.23,
  '0-1-1': 24.22,
  '0-10': 58.68,
  '0-11': 24.22,
  '00-1': 58.68}}

In [101]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19},
 'VEXYON_charged': {'-1-1-1': 553.92,
  '-1-10': 470.97,
  '-1-11': 553.92,
  '-10-1': 470.97,
  '-100': 1583.26,
  '-101': 470.97,
  '-11-1': 553.92,
  '-110': 470.96,
  '-111': 553.92,
  '0-1-1': 471.3,
  '0-10': 1583.26,
  '0-11': 470.97,
  '00-1': 1583.26}}

In [92]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19}}

In [60]:
path_dic

{'VABJIS_clean': {'-1-1-1': 5.93,
  '-1-10': 5.99,
  '-1-11': 5.91,
  '-10-1': 14.43,
  '-100': 10.74,
  '-101': 13.09,
  '-11-1': 4.91,
  '-110': 6.98,
  '-111': 6.97,
  '0-1-1': 13.09,
  '0-10': 10.74,
  '0-11': 14.43,
  '00-1': 29.19}}

In [72]:
path_dic

{'VABJIS_clean': {'-1-1-1': 10.59,
  '-1-10': 10.56,
  '-1-11': 10.56,
  '-10-1': 9.72,
  '-100': 13.01,
  '-101': 9.87,
  '-11-1': 11.49,
  '-110': 11.54,
  '-111': 11.58,
  '0-1-1': 9.89,
  '0-10': 13.04,
  '0-11': 9.73,
  '00-1': 29.19}}

In [52]:
larger_node = []
name_larger_node = []
for node in final.nodes:
    larger_node.append(final.nodes[node]['rad_max_sph'])
    name_larger_node.append(node)
node_largest = name_larger_node[np.argmax(larger_node)]
node_smallest = name_larger_node[np.argmin(larger_node)]
final.nodes[node_largest], final.nodes[node_smallest]
node_largest

20

In [10]:
5142 - key_plus*int(5142/key_plus)

176

In [21]:
a, b = dijkstra(176, 5142, {}, final)

In [24]:
b[5142]

KeyError: 5142

In [16]:
pos_order['000']

13

In [17]:
P[5142]

KeyError: 5142

In [19]:
D[5142]

inf

In [175]:



# num_cores = multiprocessing.cpu_count()
# # mat_dic = {}

# path_dic = {}

# pool = multiprocessing.Pool(num_cores)


# # Step 2: `pool.apply` the `howmany_within_range()`
# volumen_res = pool.map(Diffusion2, os.listdir('./Nt2Files_MOF')[3:5])



# # Step 3: Don't forget to close
# pool.close()


def Diffusion2(file):

    
    

    nt2_name = file

    vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
    G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

    final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
    Y, Y1 = TransfMatrix(vector_values_latt)
    final2 = distance(final,3, Y, Y1)



    neigh_dic = {}
    for n, nbrs in final2.adj.items():
        neigh = []
        for nbr, eattr in nbrs.items():
            neigh.append(nbr)


        neigh_dic[str(n)] = neigh

    all_path = []
    total_path = []
    special_list = list(range(13))

    node_path = mat_pr_dic[nt2_name.split('.')[0]]

    for location in special_list:
        parents = {}

        start = node_path + key_plus*location

        target = key_plus*pos_order['000'] + node_path

        D, P = dijkstra(start, target, parents, final2)

        path = backpedal(start, target, P)

        distance_path = []
        for i in range(len(path)-1):

            distance_path.append(final2[path[i]][path[i+1]]['distance'])


        all_path.append(path)
        total_path.append(sum(distance_path))  


    total_vol = []
    for ind_path in all_path:
        volume = []
        for i in range(len(ind_path) - 1):
            volume.append(
                final2[ind_path[i]][ind_path[i+1]]['distance']*(final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi)
        total_vol.append(sum(volume))

    path_dic_mat = {}

    for j in range(len(list(pos_order.keys())[0:13])):
        path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


    path_dic[nt2_name.split('.')[0]] = path_dic_mat
    print(path_dic)
    return path_dic

def CellLocation(nt2_name):
    
    from pymatgen.core.structure import Structure
    from math import pi, cos, sin
    
    six_letter_code = nt2_name.split('.')[0]
    
    path_nt2_file = './Nt2Files_MOF/' + nt2_name
    path_cif_file = './MOF/' + six_letter_code + '.cif'
    
    cif_structure = Structure.from_file(path_cif_file)
    
    latt = cif_structure.lattice
    
    # Angles of the cell's axis
    alpha = latt.alpha
    beta = latt.beta
    gamma = latt.gamma

    # Constant to change from degrees to radians
    cte = pi/180

    # Vectors proyections of the cell's vector over the ortogonal axis
    x_axis = latt.a
    y_axis = latt.b
    z_axis = latt.c

    # Cell's vectors
    a, b, c = round(x_axis*sin(beta*cte), 3) , 0 ,round(x_axis*cos(beta*cte), 3)
    d, e, g = round(y_axis*cos(gamma*cte), 3) ,round(y_axis*sin(gamma*cte), 3), 0
    l, m, n = 0, round(z_axis*cos(alpha*cte), 3), round(z_axis*sin(alpha*cte), 3)
    
    vector_values_latt = (a, b, c, d, e, g, l, m, n)

    # Constant that limits the maximum number of cells that could expand over every direction 
    # (axis and diagonal direction)
    r = 1

    cubic_comb = []
    
    # Each of the cell's vectors represent the length of the cell in that direction.
    # Independently of which length it is if you multiply the length of one of the vector by two it is clear
    # that the result vector will give the information about the location of the second cell in the direction
    # of the vector we multiply by two.

    # We generate the location of the cells for a cube of length equals to len(range(-r, r + 1))
    # r = 1, len(range(-r, r + 1)) = 27
    # The information of location of the cells are saved in real coordinates using the combination of the 
    # cell's vectors and in an orthogonal base transformation using as vectors the numbers (i,j,k) we use to multiply 
    # the cell's vectors
    for i in range(-r, r+1):
        for j in range(-r, r+1):
            for k in range(-r, r+1):
                cubic_comb.append((np.array([a,b,c])*i + np.array([d,e,g])*j + np.array([l,m,n])*k,
                                   np.array([i,j,k])))

    vector_pos = []
    vector_ort = []
    # Unpack values
    # We also choose as a center the vector 0 
    
    #     print('This is the Lattice vector: {}'.format(lattice_vector.abc))

    for vec, ort in cubic_comb:
        vector_pos.append(list(vec))
        vector_ort.append(list(ort))

    # Later in this Notebook we are going to create a graph for every vector we generated in the previous step.
    # These graphs will be composed by nodes whose names will be an ordered number.
    # The names will be assigned by the order the graphs were generated.
    # That means every graph will have as the name of their nodes a range of numbers whose length will be equal 
    # to the name of the last node we create in the first graph.
    # In order to keep track of the order we create we are going to establish that order by numbered the orthogonal
    # vectors

    pos_order = {}
    i = 0
    for vector in vector_ort:
        name = str(vector[0]) + str(vector[1]) + str(vector[2]) 
        pos_order[name] = i
        i += 1

    twentysix_faces_neigh = [np.array(vector_name) for vector_name in vector_ort]
    
    return vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt


def GraphCell(nt2_name, pos_order):
    
    import networkx as nx
    
    # Initialize a networkx's graph. 
    # We take the information about the nodes and edges from the .nt2 file and passes it out to the graph.

    # Every node in the graph will have three attributes.
    # -Cartesian coordinate
    # -Pagerank coefficient
    # -Radius of the maximus sphere that fit in that node

    # Evey edge will have one attribute:
    # -Radius of the maximus sphere that can travel through the edge

    # We are not going to save the information about edges that connect point from outside of the cell
    
    path_nt2_file = './Nt2Files_MOF/' + nt2_name
    
    G = nx.Graph()
    
    cell_names = list(pos_order.keys())
    
    connection_cell = {cell_name:[] for cell_name in cell_names}
    
    larger_dist_atom = []
    
    pagerank_init_value = 100
    with open(path_nt2_file) as f:
        line = ' '

        while line:
            line = f.readline()

            if line == 'Vertex table:\n': 

                while line:

                    line = f.readline()

                    if line == '\n':
                        break
                    else:
                        line_list = line.split()
                        key = int(line_list[0])
                        coord_x = float(line_list[1])
                        coord_y = float(line_list[2])
                        coord_z = float(line_list[3])
                        min_dist_atom = float(line_list[4])

                        G.add_node(key, coord=np.array([coord_x, coord_y, coord_z]), rad_max_sph=min_dist_atom,
                                  pagerank=pagerank_init_value)
                        larger_dist_atom.append(min_dist_atom)

            if line == 'Edge table:\n':


                while line:

                    line = f.readline()
                    if line == '':
                        break
                    line_list = line.split()
                    
                    origin = int(line_list[0])
                    destination = int(line_list[2])
                    
                    larger_radius = float(line_list[3])
                    
                    x_sim = int(line_list[4])
                    y_sim = int(line_list[5])
                    z_sim = int(line_list[6])
                    
                    identifier = str(x_sim) + str(y_sim) + str(z_sim)
                    
                    if sum([abs(x_sim), abs(y_sim), abs(z_sim)]) == 0:
                        G.add_edge(origin, destination, rad_max_sph=larger_radius)
                        

                    else:
                        connection_cell[identifier].append((origin,destination,larger_radius))

    
    larger_node = max(larger_dist_atom)
    index_larger_node = np.argmax(larger_dist_atom)
    name_larger_node = list(G.nodes)[index_larger_node]
    
    return G, connection_cell, {'LargerNode':larger_node, 'NameLargerNode':name_larger_node} 


def GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh):
    
    import networkx as nx
    from copy import deepcopy
    from networkx.relabel import relabel_nodes
    # Create a dictionary which contains one graph for every cell we want to use to calculate the 
    # pagerank coefficients. 
    # These new graphs are a copy of the one that we initialized in previous step so, we 
    # change the names and the cartesian coordinates for every node

    graph_mod= {}

    # As we mentioned earlier we are goint to name the nodes wit a number.
    # The name of the last node of the first graph plus one (because we start in 0) will give as the range in every 
    # graph.

    key_plus = sorted(list(G.nodes), reverse=True)[0] + 1

    # The names used as keys in the dictionary of graph are the ortogonal base transformations we calculated before
    # They names are very useful to know the position of the graph with respect to the origin (0,0,0).

    for i in range(len(vector_pos)):
        name = str(vector_ort[i][0])+str(vector_ort[i][1])+str(vector_ort[i][2])
        graph_mod[name] = deepcopy(G)

        mapping_name = {}
        multiplier = pos_order[name]
        for node in graph_mod[name].nodes:
            graph_mod[name].nodes[node]['coord'] += vector_pos[i] #*box
            mapping_name[node] = node + key_plus*multiplier
        relabel_nodes(graph_mod[name], mapping=mapping_name, copy=False)

    # As you can see here it is really to see where the graph is if we now their name
    #     print('Keys of the Dictionary of Graph: {}'.format(graph_mod.keys()))


    # List with the names of the graph
    
    def symm(array):
    
        import numpy as np
        first = np.where(array == 2, -1, array)
        second = np.where(first == -2, 1, first)

        return second 

    # Now we add to the graph the edges that connect with nodes outside of the cell. We do this for every graph 
    # in the dictionary

    for j in range(len(vector_pos)):
        for neighbor in twentysix_faces_neigh:
            str_neighbor = str(neighbor[0]) + str(neighbor[1]) + str(neighbor[2]) 

            neig_loc = vector_ort[j] + neighbor
            str_neig_loc = str(neig_loc[0])+str(neig_loc[1])+str(neig_loc[2])

            if str_neig_loc in list(pos_order.keys()):
                name = str(vector_ort[j][0])+str(vector_ort[j][1])+str(vector_ort[j][2])
                multiplier_origin = pos_order[name]
                multiplier_destination = pos_order[str_neig_loc]

                for edge in connection_cell[str_neighbor]:
                    edge_1 = edge[0] + multiplier_origin*key_plus
                    edge_2 = edge[1] + multiplier_destination*key_plus
                    rad_max_sph = edge[2]
                    graph_mod[name].add_edge(edge_1, edge_2)
                    graph_mod[name].edges[edge_1, edge_2]['rad_max_sph'] = rad_max_sph

            else:
                symm_vec = symm(neig_loc)

                name = str(vector_ort[j][0])+str(vector_ort[j][1])+str(vector_ort[j][2])
                str_neig_loc_symm = str(symm_vec[0])+str(symm_vec[1])+str(symm_vec[2])

                multiplier_origin = pos_order[name]
                multiplier_destination = pos_order[str_neig_loc_symm]

                for edge in connection_cell[str_neighbor]:
                    edge_1 = edge[0] + multiplier_origin*key_plus
                    edge_2 = edge[1] + multiplier_destination*key_plus
                    rad_max_sph = edge[2]
                    graph_mod[name].add_edge(edge_1, edge_2)
                    graph_mod[name].edges[edge_1, edge_2]['rad_max_sph'] = rad_max_sph

    final = nx.Graph()

    for key in graph_mod:
        final.add_nodes_from(graph_mod[key])
        final.add_edges_from(graph_mod[key].edges)
        for node in graph_mod[key].nodes:
            for attr in graph_mod[key].nodes[node]:
                final.nodes[node][attr] = graph_mod[key].nodes[node][attr]

        for edge in graph_mod[key].edges:
            for attr in graph_mod[key].edges[edge[0], edge[1]]:
                final.edges[edge][attr] = graph_mod[key].edges[edge][attr]
                
    return final, key_plus

def TransfMatrix(vector_values_latt):
    import numpy as np
    a,b,c,d,e,g,l,m,n = vector_values_latt
    # Matrix to change the orthogonal base vector to the base vector of the unit cell

    A = np.array([[a,b,c],[d,e,g], [l,m,n]]).transpose()
    B1 = np.array([1, 0, 0])
    B2 = np.array([0, 1, 0])
    B3 = np.array([0, 0, 1])
    X = np.array([np.linalg.inv(A).dot(B1),np.linalg.inv(A).dot(B2),np.linalg.inv(A).dot(B3)])
    Y = X.transpose()

    # Matrix to change the base vector of the unit cell for the orthogonal base vector
    A1 = np.array([[1,0,0], [0,1,0], [0,0,1]]).transpose()
    B4 = np.array([a,b,c])#*3
    B5 = np.array([d,e,g])#*3
    B6 = np.array([l,m,n])#*3
    X1 = np.array([np.linalg.inv(A1).dot(B4),np.linalg.inv(A1).dot(B5),np.linalg.inv(A1).dot(B6)])
    Y1 = X1.transpose()

    return Y, Y1

def distance(final, multi_box, Y, Y1):
    # Function for the calculation of distances between axes
    import numpy as np
    dim = 3
    real_box = np.array([multi_box,multi_box,multi_box])
    
    for edge in final.edges:
        p1 = Y.dot(final.nodes[edge[0]]['coord'])
        p2 = Y.dot(final.nodes[edge[1]]['coord'])
    #     print(p1, p2)
        dist_3d = np.zeros((dim))
        for j in range(dim):
            dist = abs(p2[j]-p1[j])
            if dist > real_box[j] * 0.5:
                dist = abs(real_box[j] - dist)
            dist_3d[j] = dist
        
        final[edge[0]][edge[1]]['distance'] = np.sqrt(np.sum(np.power(Y1.dot(dist_3d), 2)))
             #- (final.nodes[edge[0]]['rad_max_sph'] + final.nodes[edge[1]]['rad_max_sph'])
    return final

# for edge in final.edges:
#     final[edge[0]][edge[1]]['distance'] = distance(final, edge[0], edge[1], 3, Y, Y1)



def dijkstra(start_vertex, target, parents, final, tol):
    from queue import PriorityQueue
    
    # Dictionary with keys = vertices and values = weights of the vertices (set as infinity at the start)
    D = {}
    visited = []
    node_list = list(final.nodes)

    D = {v:float('inf') for v in node_list}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        if current_vertex == target:
            print('Target vertex: ' + str(target) + ' reached')
            break
        else:

            visited.append(current_vertex)


            # Loop for every vertex of the net
            for neighbor in neigh_dic[str(current_vertex)]:
#                         if self.edges[current_vertex][neighbor] != -1:
               
                distance_edge = final[current_vertex][neighbor]['distance']

                if neighbor not in visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance_edge
                    if (new_cost < old_cost and (final.nodes[neighbor]['rad_max_sph']>tol and final[current_vertex][neighbor]['rad_max_sph']>tol)):
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost
                        parents[neighbor] = current_vertex

    return D, parents

def backpedal(start_vertex, target, searchResult):

    node = target

    backpath = [node]

    path = []

    while node != start_vertex:

        backpath.append(searchResult[node])

        node = searchResult[node]

    for i in range(len(backpath)):

        path.append(backpath[-i - 1])

    return path 



In [173]:
from queue import PriorityQueue
start_vertex = 176
target = 5142
parents = {}
# Dictionary with keys = vertices and values = weights of the vertices (set as infinity at the start)
D = {}
visited = []
node_list = list(final.nodes)

D = {v:float('inf') for v in node_list}
D[start_vertex] = 0

pq = PriorityQueue()
pq.put((0, start_vertex))

while not pq.empty():
    (dist, current_vertex) = pq.get()
    if current_vertex == target:
        print('Target vertex: ' + str(target) + ' reached')
        break
    else:
        print(current_vertex)
        visited.append(current_vertex)


        # Loop for every vertex of the net
        for neighbor in neigh_dic[str(current_vertex)]:
#                         if self.edges[current_vertex][neighbor] != -1:

            distance_edge = final[current_vertex][neighbor]['distance']

            if neighbor not in visited:
                old_cost = D[neighbor]
                new_cost = D[current_vertex] + distance_edge
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor))
                    D[neighbor] = new_cost
                    parents[neighbor] = current_vertex


176
435
177
178
433
208
205
296
299
180
334
181
295
210
181
426
444
298
309
378
211
179
318
317
297
186
188
301
276
431
434
419
96
213
394
413
1657
320
321
335
1654
170
279
187
288
420
23
395
1663
289
418
377
442
189
427
300
423
19
21
294
383
336
382
393
377
311
443
315
23
30
336
277
190
1651
90
332
1675
1347
25
384
21
1591
384
332
430
304
424
333
429
35
235
341
344
1649
390
380
1662
1421
1659
388
171
1359
238
1366
1659
283
389
407
410
282
269
425
425
1350
1466
1695
432
286
1352
1434
436
1369
82
1357
441
1375
1415
1382
233
400
93
1694
284
1387
1399
1697
1400
441
1665
285
232
282
1661
1404
22
1702
373
93
1704
305
27
1396
389
374
265
95
1348
360
428
16
1696
362
13
146
246
1383
163
404
1437
1351
306
1371
375
232
385
1470
1642
244
376
1401
398
260
381
327
7
1648
361
201
308
1472
27
1596
33
331
142
141
1357
162
1775
244
405
200
133
81
147
357
339
268
353
379
412
144
408
204
1658
292
411
402
1351
1348
95
1411
92
95
84
83
339
396
273
260
345
91
1527
1395
1377
1439
1776
1757
1449
1386
202
1407

1297
1607
9353
1612
2659
9617
2787
9675
2030
9773
2152
2347
4209
4181
8186
545
1783
8446
486
3008
8301
2273
8608
2009
5335
9656
4085
2291
829
2348
552
1077
4262
1071
2932
532
3144
4907
4301
6856
8187
1475
8344
8496
8302
880
1325
4107
4072
1529
1315
1123
1101
4163
572
9611
2986
1326
4367
8176
1783
8102
1501
9824
8116
8738
2882
1559
1309
1425
4056
571
8177
4020
911
1502
2743
1462
2741
2926
4140
9921
2730
4019
4138
8115
3071
5308
9821
1727
3010
2682
8844
8862
8360
9723
8865
8737
2668
8724
8880
2961
2726
458
4075
1461
8739
2721
9355
611
743
8455
2737
538
778
872
746
2676
4188
8388
1749
2768
4376
8361
2933
4375
1485
4018
612
4440
8445
8137
1131
551
8536
620
721
4022
4333
6704
4203
9889
9791
2885
6945
530
8460
550
786
868
8136
8158
1994
2963
985
2748
8740
756
4027
2740
1301
8300
1273
4205
4298
8737
3142
2896
4339
1639
9628
488
9769
3240
8354
2323
74
1322
8288
9902
4027
8445
4253
8360
8366
1622
8808
3148
9734
9752
763
9755
9627
9942
2146
9614
9770
1067
1047
2891
3170
8269
3111
4142
835
825
40

1998
3559
1332
4174
8109
11456
1247
5337
8647
8515
1827
8238
8497
8559
4245
5414
10082
9425
8121
3359
9572
3320
8569
8506
2029
9763
4309
5700
8122
8353
937
3370
8745
8708
10685
3509
3492
2406
1209
1195
1249
9634
3387
8553
9812
10082
2147
3625
10145
4342
1910
4425
8889
7755
3286
4991
9882
902
8237
4387
8123
9878
8733
8144
3493
9663
1212
4337
2476
2436
8059
2149
6486
6888
3624
5404
8304
8347
4918
9484
1194
10121
1999
9449
4388
5401
9506
11289
4348
9669
6979
974
10711
1844
8141
8782
7899
9662
7752
3845
8064
3271
10766
9887
5405
3330
9397
8149
9658
2728
8114
3479
2559
8636
6412
2173
11094
8856
9434
9930
9878
10167
9430
8474
9487
11021
1845
8596
2034
8468
5266
8403
9804
3765
8476
1812
11076
8718
2463
1931
8816
2656
5716
4012
1241
6887
10006
5388
1829
5262
4250
11214
2477
9524
3294
8636
5692
10148
3487
4343
8802
3768
8846
665
1808
602
11076
1803
9454
9964
1825
5654
1062
5416
2011
10009
9660
2572
2616
3844
4914
11013
10855
884
6746
1834
973
2404
5715
5289
3315
9548
8850
8453
896
3503
8110
334

11394
11550
10778
1863
2025
3133
10214
4390
10996
4010
10792
8082
9998
9400
898
1111
4052
1058
3895
9997
5437
11534
11080
10234
10687
3138
3882
4409
10954
7983
3115
3208
2545
6629
4675
3864
979
3251
975
7098
8007
11042
11036
2294
3983
6810
3534
10793
10688
1958
1028
9995
1944
10208
3881
10088
8065
2560
9999
9595
9872
3879
2131
8457
10079
3937
3115
5481
10915
1952
10827
7016
10099
4358
5124
2605
10769
10696
8818
4093
11020
10692
6948
10033
10813
6846
4673
11431
10723
2075
9380
2525
9473
10771
7943
10762
3211
10075
5158
2607
7830
4344
4099
6901
10013
7177
10025
3987
6777
2143
7059
4237
10137
10839
4403
10181
10072
1007
9622
1263
9388
4356
10702
11253
10764
9535
3305
9715
2576
10850
4249
3993
2213
1932
11156
4386
2513
3600
3328
4123
6836
917
2632
11211
7173
2286
5733
1961
7842
10015
4459
8046
3879
6591
4088
10753
9872
2415
2369
5315
10740
4407
1868
2657
3311
4120
1264
6896
3607
1941
9620
3857
2232
11022
2301
3137
6943
4050
3347
8228
3272
5277
2231
8326
5227
2557
11436
2151
2398
10780
1103

7807
11171
5226
11229
3443
3581
9858
5447
4257
11556
9845
11298
5683
7954
7972
1120
11267
2127
2171
5703
3939
7975
6841
11276
4951
1900
6982
2664
11363
2235
11046
5247
4904
1118
7834
7990
942
3622
1096
945
927
894
6899
10822
11295
8826
1113
3333
10776
11200
7942
10031
2439
5440
2067
6768
11459
6417
935
6820
6842
5240
5155
6624
6889
5520
4162
6744
899
2428
2606
959
8006
1863
5543
1018
5368
10330
4108
999
3619
1246
4482
928
11168
4444
10881
2512
6800
11411
4976
11399
11155
1115
5125
5042
7153
2503
3161
10199
10703
9138
6683
5703
11323
929
11032
5370
5187
1126
3875
10808
10328
3640
919
5517
10857
5635
3865
11120
5039
5040
6968
3837
2032
1088
6602
4973
4224
970
6706
6695
3334
921
4909
7002
1868
6985
5129
10527
6553
8503
6430
6675
7588
6962
3991
5039
7595
3590
5203
2533
959
11131
5128
7032
3474
7984
7028
7054
9063
6306
1885
10065
3396
11542
3654
2306
3994
4656
969
4978
7985
3130
5295
8946
8825
4483
10776
5538
11132
3999
6491
3821
1905
11258
4975
3223
4671
2254
5109
2228
3132
10820
3127
1070

6404
9242
6608
11173
9127
6663
11558
8918
11364
10579
7961
7514
5295
9319
10245
3297
9130
3465
10342
5745
5045
4940
4514
4823
11141
1147
8005
1109
3904
3817
6965
5608
5141
7670
9141
6660
7893
3603
1215
5176
6480
10969
10489
7979
4507
11241
9000
9128
9057
6663
4716
2136
9258
11425
2462
11161
11558
9039
10335
5790
4515
3630
9134
10397
7767
6798
7580
5003
4934
7926
7577
6953
7763
10866
5154
949
5438
5019
7671
5165
5933
2390
11323
9003
11314
2378
8914
9007
9864
11569
5135
7946
7113
6833
5101
10479
3641
2443
11268
8983
2567
6327
6300
6797
6574
6914
5381
4826
3615
2468
5348
10440
11561
9021
4498
3593
3909
6834
3244
5135
3592
3710
3667
9310
2500
6539
3955
10650
5526
4944
7839
5529
10886
4822
4614
10443
10331
9105
5431
6233
6791
6608
11188
9011
4825
10647
5345
5717
8611
5513
5047
5515
4650
3692
3852
7408
4643
3862
11491
3660
10498
4949
6244
6794
5952
11737
5131
9283
11191
9139
11426
3163
9108
3670
10204
8996
9025
3631
5476
5634
2328
5547
5831
10639
2245
10632
11335
4178
3822
3562
9305
7798
758

In [26]:
final[5142]

AtlasView({4768: {'rad_max_sph': 0.605995, 'distance': 0.4189999534779901}, 4951: {'rad_max_sph': 0.592359, 'distance': 0.8497313702011949}, 5140: {'rad_max_sph': 0.751209, 'distance': 0.7386831728758955}, 5152: {'rad_max_sph': 0.791739, 'distance': 0.7378504479357471}})

In [ ]:
final[edge[0]][edge[1]]['distance'] = np.sqrt(np.sum(np.power(Y1.dot(dist_3d), 2)))

In [32]:
(final.nodes[1]['rad_max_sph']**3)*(4/3)*pi

0.05225609141437991

In [40]:
((final[1][5]['distance'] - final.nodes[1]['rad_max_sph'] - final.nodes[5]['rad_max_sph'])*
 (final[1][5]['rad_max_sph']**2)*pi) + (final.nodes[1]['rad_max_sph']**3)*(4/3)*pi +(final.nodes[5]['rad_max_sph']**3)*(4/3)*pi

0.11261776320089582

In [41]:
((final[1][5]['distance'])*(final[1][5]['rad_max_sph']**2)*pi)

0.0659781074193123